# 考虑以contextlib和with语句来改写可复用的try/finally代码

In [1]:
import logging
from pprint import pprint

**示例：**如果把互斥锁放在with语句之中，就表示：只有当程序持有该锁的时候，with语句块里的那些代码，才会得到运行。

In [2]:
from threading import Lock
lock = Lock()
with lock:
    print('Lock is held')

Lock is held


In [3]:
# 这种写法与上面的效果是相似的
lock.acquire()
try:
    print('Lock is held')
finally:
    lock.release()

Lock is held


## 使用contextlib模块来处理自己所编写的对象和函数，使它们能够支持with语句

**示例：**当程序运行到某一部分时，我们希望针对这部分代码，打印出更为详细的调试信息。

In [4]:
logging.getLogger().setLevel(logging.WARNING)

In [5]:
def my_function():
    logging.debug('Some debug data')
    logging.error('Error log here')
    logging.debug('More debug data')

In [6]:
my_function()

ERROR:root:Error log here


**改进：**定义一种情境管理器，来临时提升函数的信息级别。在运行with块内的代码之前，临时提升信息级别，待with块执行完毕，再恢复原有级别。

In [7]:
from contextlib import contextmanager
@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield
    finally:
        logger.setLevel(old_level)

In [8]:
with debug_logging(logging.DEBUG):
    print('Inside:')
    my_function()
print('After:')
my_function()

DEBUG:root:Some debug data
ERROR:root:Error log here
DEBUG:root:More debug data
ERROR:root:Error log here


Inside:
After:


## 使用带有目标的with语句

**示例1：**向文件中写入数据，并且确保该文件总是能正确地关闭。

In [9]:
with open('my_output.txt', 'w') as handle:
    handle.write('This is some data!')

**示例2：**定义一个情境管理器，能够获取Logger实例、设置其级别，并通过yield语句将其提供给由as关键字所指定的目标。

In [10]:
@contextmanager
def log_level(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield logger
    finally:
        logger.setLevel(old_level)

In [11]:
with log_level(logging.DEBUG, 'my-log') as logger:
    logger.debug('This is my message!')
    logging.debug('This will not print')

DEBUG:my-log:This is my message!


由于with语句块可以把严重级别调低，所以在as目标变量上面调用debug等方法，可以打印出DEBUG级别的调试信息。若直接在logging模块上面调用debug，则不会打印出任何DEBUG级别的消息。

In [12]:
logger = logging.getLogger('my-log')
logger.debug('Debug will not print')
logger.error('Error will print')

ERROR:my-log:Error will print


退出with语句块之后，在名为'my-log'的Logger上面调用debug方法，打印不出消息。因为Logger的严重级别已经恢复到默认的WARNING。但由于ERROR级别高于WARNING级别，所以能打印出消息。